In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')



In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./Defense_Distillation.csv", mode='a', index=False)




In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
epochs = 50
training_batch_size = 128
validation_batch_size = 128
learning_rate = 0.01
baseline_temperature = 1
distilled_temperature = 100


x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)


input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize models
teacher_model = DNNModel(input_shape, output_shape).to(device)
student_model = DNNModel(input_shape, output_shape).to(device)


In [5]:
# Training function for the teacher model
def train_teacher_model(model, train_loader, epochs, temperature):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_loss = 0.0
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs / temperature, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(model.state_dict(), 'teacher_model.pth')



In [6]:
# Train the teacher model
# train_teacher_model(teacher_model, train_loader, epochs, baseline_temperature)

In [7]:
teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Defense_Distillation/teacher_model.pth"))

/tmp/ipykernel_182/2105613291.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Def

<All keys matched successfully>

In [8]:
# Distillation loss function
def distillation_loss(student_outputs, teacher_outputs, temperature, alpha):
    soft_targets = F.softmax(teacher_outputs / temperature, dim=1)
    student_log_probs = F.log_softmax(student_outputs / temperature, dim=1)
    distillation_loss = nn.KLDivLoss()(student_log_probs, soft_targets) * (temperature ** 2)
    return distillation_loss

# Training function for the student model using distillation
def train_student_model(student_model, teacher_model, train_loader, epochs, temperature, alpha=0.5):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')
    
    student_model.train()
    teacher_model.eval()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    
    train_loss = 0.0

    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            teacher_outputs = teacher_model(inputs).detach()
            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        student_model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                teacher_outputs = teacher_model(inputs).detach()
                student_outputs = student_model(inputs)
                loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
                val_train_loss += loss.item()
                _, predicted = torch.max(student_outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(student_model.state_dict(), 'student_model.pth')



In [9]:
# Train the student model
# train_student_model(student_model, teacher_model, train_loader, epochs, distilled_temperature)


In [10]:
student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Defense_Distillation/student_model.pth"))

/tmp/ipykernel_182/3271045883.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Def

<All keys matched successfully>

In [11]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)




In [12]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def8/y_pred_{attack_name}{eps}_Def8.npy", all_preds)


In [13]:

# calculate_performance_metrics(x_test, y_test, student_model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)

In [12]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(211, 56) (211,)
Accuracy: 0.20853080568720378

macro
Precision: 0.36992481203007516
Recall: 0.4255380200860832
F1 Score: 0.20135536365902856

weighted
Precision: 0.4931190535580658
Recall: 0.20853080568720378
F1 Score: 0.15507376257729846

start XGB 100 BIM
(119, 56) (119,)
Accuracy: 0.3697478991596639

macro
Precision: 0.38873873873873876
Recall: 0.3905141843971631
F1 Score: 0.36956982411527867

weighted
Precision: 0.40511015216897567
Recall: 0.3697478991596639
F1 Score: 0.3673438860604636

start XGB 100 FGSM
(732, 56) (732,)
Accuracy: 0.04371584699453552

macro
Precision: 0.1419216207919779
Recall: 0.15282738095238094
F1 Score: 0.04370870814825874

weighted
Precision: 0.23948969475956677
Recall: 0.04371584699453552
F1 Score: 0.04152422118756543

start XGB 100 PGD
(119, 56) (119,)
Accuracy: 0.3697478991596639

macro
Precision: 0.38873873873873876
Recall: 0.3905141843971631
F1 Score: 0.36956982411527867

weighted
Precision: 0.40511015216897567
Recall: 0.36974789

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8975859546452085

macro
Precision: 0.5
Recall: 0.44879297732260426
F1 Score: 0.47301464919043945

weighted
Precision: 1.0
Recall: 0.8975859546452085
F1 Score: 0.9460292983808789

start XGB 1 PGD
(139, 56) (139,)
Accuracy: 0.2589928057553957

macro
Precision: 0.49096548004314994
Recall: 0.3777372262773723
F1 Score: 0.21183594825213323

weighted
Precision: 0.9583731073393711
Recall: 0.2589928057553957
F1 Score: 0.3990764118680283

start XGB 1 DF
(14541, 56) (14541,)
Accuracy: 0.15500997180386494

macro
Precision: 0.4487460046436921
Recall: 0.11727345028820917
F1 Score: 0.13502989769180967

weighted
Precision: 0.8783314949651595
Recall: 0.15500997180386494
F1 Score: 0.2613925655036632

start XGB 1 AutoPGD
(92, 56) (92,)
Accuracy: 0.15217391304347827

macro
Precision: 0.5125
Recall: 0.5666666666666667
F1 Score: 0.14203730272596843

weighted
Precision: 0.9788043478260869
Recall: 0.15217391304347827
F1 Score: 0.2312394735200549

start XGB 1 ZOO
(116, 56) (116,)
Accuracy: 0.301724

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.043478260869565216

macro
Precision: 0.24243565599497802
Recall: 0.05450597176981542
F1 Score: 0.04214796214023566

weighted
Precision: 0.45921013128087557
Recall: 0.043478260869565216
F1 Score: 0.07451856455392161

start RF 100 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/x_test_adv_BIM_Def8.npy not found
start RF 100 FGSM
(67, 56) (67,)
Accuracy: 0.029850746268656716

macro
Precision: 0.016129032258064516
Recall: 0.14285714285714285
F1 Score: 0.028985507246376812

weighted
Precision: 0.003370245546461242
Recall: 0.029850746268656716
F1 Score: 0.006056673155959334

start RF 100 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input100/x_test_adv_PGD_Def8.npy not found
start RF 100 DF
(18058, 56) (18058,)
Accuracy: 0.1987484771292502

macro
Precision: 0.4799928757922072
Recall: 0.37669651664732035
F1 Score: 0.17514025731774024

weighted
Precision: 0.9174273558795113
Recall: 0.1987484771292502
F1 

...

Accuracy: 0.1897496318114875

macro
Precision: 0.47149292666161596
Recall: 0.3252611540258955
F1 Score: 0.16693773624836217

weighted
Precision: 0.90431323119658
Recall: 0.1897496318114875
F1 Score: 0.2976452688537385

start RF 50 AutoPGD
(708, 56) (708,)
Accuracy: 0.0423728813559322

macro
Precision: 0.20833333333333334
Recall: 0.02252252252252252
F1 Score: 0.04065040650406504

weighted
Precision: 0.3919491525423729
Recall: 0.0423728813559322
F1 Score: 0.07647788342290203

start RF 50 ZOO
(755, 56) (755,)
Accuracy: 0.10331125827814569

macro
Precision: 0.28332128845756765
Recall: 0.25308823529411767
F1 Score: 0.10308467908212417

weighted
Precision: 0.4707671890746938
Recall: 0.10331125827814569
F1 Score: 0.11450804688154312

start RF 50 CaFA
(99, 56) (99,)
Accuracy: 0.9292929292929293

macro
Precision: 0.8777660695468914
Recall: 0.9377289377289377
F1 Score: 0.9023531069465971

weighted
Precision: 0.9402560909410226
Recall: 0.9292929292929293
F1 Score: 0.9318832968262305

start RF 50 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.1588235294117647

macro
Precision: 0.27124183006535946
Recall: 0.35833333333333334
F1 Score: 0.1538756047474853

weighted
Precision: 0.36216839677047286
Recall: 0.1588235294117647
F1 Score: 0.11200854989589028

start RF 20 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input20/x_test_adv_BIM_Def8.npy not found
start RF 20 FGSM
(204, 56) (204,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

start RF 20 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input20/x_test_adv_PGD_Def8.npy not found
start RF 20 DF
(16069, 56) (16069,)
Accuracy: 0.1817785798742921

macro
Precision: 0.46042485723147286
Recall: 0.298042458309502
F1 Score: 0.1613991006095252

weighted
Precision: 0.877661211979647
Recall: 0.1817785798742921
F1 Score: 0.28410715879334925

start RF 20 AutoPGD
(438, 56) (438,)
Accuracy: 0.0273972602739726

macro
Precision: 0.15384615384615385
Recall: 0.014598540145985401
F1 Score: 0.02666666666666667

weighted
Precision: 0.28872497365648053
Recall: 0.0273972602739726
F1 Score: 0.05004566210045663

start RF 20 ZOO
(510, 56) (510,)
Accuracy: 0.1588235294117647

macro
Precision: 0.27124183006535946
Recall: 0.35833333333333334
F1 Score: 0.1538756047474853

weighted
Precision: 0.36216839677047286
Recall: 0.1588235294117647
F1 Score: 0.11200854989589026

start RF 20 CaFA
(101, 56) (101,)
Accuracy: 0.9207920792079208

macro
Precision: 0.91765873

In [10]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(176, 56) (176,)
Accuracy: 0.23295454545454544

macro
Precision: 0.37587904360056257
Recall: 0.4339322478008609
F1 Score: 0.22187653512362862

weighted
Precision: 0.47592619230277455
Recall: 0.23295454545454544
F1 Score: 0.17017915357935007

start XGB 100 BIM
(97, 56) (97,)
Accuracy: 0.422680412371134

macro
Precision: 0.37039800995024874
Recall: 0.38574561403508767
F1 Score: 0.37419354838709673

weighted
Precision: 0.3944196543057907
Recall: 0.422680412371134
F1 Score: 0.4047223145992683

start XGB 100 FGSM
(783, 56) (783,)
Accuracy: 0.34099616858237547

macro
Precision: 0.4683908045977011
Recall: 0.4266554261189454
F1 Score: 0.3055074257425743

weighted
Precision: 0.7685735676223192
Recall: 0.34099616858237547
F1 Score: 0.42881576950798533

start XGB 100 PGD
(97, 56) (97,)
Accuracy: 0.422680412371134

macro
Precision: 0.37039800995024874
Recall: 0.38574561403508767
F1 Score: 0.37419354838709673

weighted
Precision: 0.3944196543057907
Recall: 0.422680412371134
F

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.11428571428571428

macro
Precision: 0.06779661016949153
Recall: 0.21052631578947367
F1 Score: 0.10256410256410256

weighted
Precision: 0.036803874092009685
Recall: 0.11428571428571428
F1 Score: 0.055677655677655674

start XGB 50 VNIFGSM
(57, 56) (57,)
Accuracy: 0.22807017543859648

macro
Precision: 0.325
Recall: 0.13
F1 Score: 0.18571428571428572

weighted
Precision: 0.5701754385964912
Recall: 0.22807017543859648
F1 Score: 0.32581453634085217

start XGB 20 baseline
(149, 56) (149,)
Accuracy: 0.2483221476510067

macro
Precision: 0.3670212765957447
Recall: 0.4638030888030888
F1 Score: 0.21872659176029963

weighted
Precision: 0.4339568756247323
Recall: 0.2483221476510067
F1 Score: 0.14218636100847096

start XGB 20 BIM
(58, 56) (58,)
Accuracy: 0.27586206896551724

macro
Precision: 0.32546583850931676
Recall: 0.2598290598290598
F1 Score: 0.2536764705882353

weighted
Precision: 0.4577425572927821
Recall: 0.27586206896551724
F1 Score: 0.3246703853955375

start XGB 20 FGSM
(712, 56

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.04

macro
Precision: 0.021212121212121213
Recall: 0.20588235294117646
F1 Score: 0.038461538461538464

weighted
Precision: 0.004121212121212122
Recall: 0.04
F1 Score: 0.0074725274725274725

start RF 100 baseline
(321, 56) (321,)
Accuracy: 0.040498442367601244

macro
Precision: 0.23246414602346807
Recall: 0.052941176470588235
F1 Score: 0.039370384764865915

weighted
Precision: 0.44012964700435003
Recall: 0.040498442367601244
F1 Score: 0.0692126358917733

start RF 100 BIM
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 100 FGSM
(21, 56) (21,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 100 PGD
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 100 DF
(17914, 56) (17914,)


...

Accuracy: 0.1955537258130918

macro
Precision: 0.4737713687566386
Recall: 0.34704963878841677
F1 Score: 0.17234929532955204

weighted
Precision: 0.9047980460322541
Recall: 0.1955537258130918
F1 Score: 0.30325423315748756

start RF 50 AutoPGD
(739, 56) (739,)
Accuracy: 0.028416779431664412

macro
Precision: 0.15671641791044777
Recall: 0.015151515151515152
F1 Score: 0.02763157894736842

weighted
Precision: 0.2939228081513946
Recall: 0.028416779431664412
F1 Score: 0.05182323196353536

start RF 50 ZOO
(783, 56) (783,)
Accuracy: 0.09578544061302682

macro
Precision: 0.2476190476190476
Recall: 0.30502759043531574
F1 Score: 0.09513456340345516

weighted
Precision: 0.38974639664294836
Recall: 0.09578544061302682
F1 Score: 0.0760731594088567

start RF 50 CaFA
(43, 56) (43,)
Accuracy: 0.9302325581395349

macro
Precision: 0.4878048780487805
Recall: 0.47619047619047616
F1 Score: 0.4819277108433735

weighted
Precision: 0.952921157118548
Recall: 0.9302325581395349
F1 Score: 0.9414401793219389

start

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.25153374233128833

macro
Precision: 0.3658940397350993
Recall: 0.4514194322271091
F1 Score: 0.22706778606965172

weighted
Precision: 0.43253565189127696
Recall: 0.25153374233128833
F1 Score: 0.15873183926990814

start RF 20 BIM
(24, 56) (24,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 20 FGSM
(32, 56) (32,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 20 PGD
(24, 56) (24,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 20 DF
(16314, 56) (16314,)
Accuracy: 0.1779453230354297

macro
Precision: 0.4592733330529641
Recall: 0.29162504509004133
F1 Score: 0.15822508520945625

weighted
Precision: 0.8765293695824534
Recall: 0.1779453230354297
F1 Score: 0.2792948672968303

start RF 20 AutoPGD
(406, 56) (406,)
Accuracy: 0.03694581280788178

macro
Precision

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.13875562772160308

macro
Precision: 0.465783584232439
Recall: 0.20474802704598224
F1 Score: 0.12376802942185125

weighted
Precision: 0.9134262850324207
Recall: 0.13875562772160308
F1 Score: 0.235016268945912

start RF 1 AutoPGD
(54, 56) (54,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 1 ZOO
(69, 56) (69,)
Accuracy: 0.21739130434782608

macro
Precision: 0.17857142857142858
Recall: 0.17857142857142858
F1 Score: 0.17857142857142858

weighted
Precision: 0.21739130434782608
Recall: 0.21739130434782608
F1 Score: 0.21739130434782608

start RF 1 CaFA
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input1/x_test_adv_CaFA_Def8.npy not found
start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input1/x_test_adv_SINIFGSM_Def8.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynami

In [10]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(23, 56) (23,)
Accuracy: 0.17391304347826086

macro
Precision: 0.20535714285714285
Recall: 0.20535714285714285
F1 Score: 0.17391304347826086

weighted
Precision: 0.23680124223602483
Recall: 0.17391304347826086
F1 Score: 0.17391304347826086

start XGB 100 BIM
(612, 56) (612,)
Accuracy: 0.29901960784313725

macro
Precision: 0.43434409393662365
Recall: 0.20204741379310345
F1 Score: 0.23503343385341552

weighted
Precision: 0.8166008897058533
Recall: 0.29901960784313725
F1 Score: 0.4331375205560569

start XGB 100 FGSM
(1767, 56) (1767,)
Accuracy: 0.7068477645727221

macro
Precision: 0.57564789875341
Recall: 0.732339190073003
F1 Score: 0.552837433802982

weighted
Precision: 0.9095908050866678
Recall: 0.7068477645727221
F1 Score: 0.773976664773226

start XGB 100 PGD
(612, 56) (612,)
Accuracy: 0.29901960784313725

macro
Precision: 0.43434409393662365
Recall: 0.20204741379310345
F1 Score: 0.23503343385341552

weighted
Precision: 0.8166008897058533
Recall: 0.29901960784313

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.14285714285714285

macro
Precision: 0.13333333333333333
Recall: 0.12878787878787878
F1 Score: 0.13098495212038302

weighted
Precision: 0.14829931972789115
Recall: 0.14285714285714285
F1 Score: 0.1454954074653117

start XGB 1 baseline
(53, 56) (53,)
Accuracy: 0.49056603773584906

macro
Precision: 0.4219858156028369
Recall: 0.4685714285714285
F1 Score: 0.38451612903225807

weighted
Precision: 0.4270038806369597
Recall: 0.49056603773584906
F1 Score: 0.39897748021911145

start XGB 1 BIM
(152, 56) (152,)
Accuracy: 0.35526315789473684

macro
Precision: 0.3153409090909091
Recall: 0.27489177489177485
F1 Score: 0.2902611015818563

weighted
Precision: 0.4284539473684211
Recall: 0.35526315789473684
F1 Score: 0.38635109787046235

start XGB 1 FGSM
(220, 56) (220,)
Accuracy: 0.15454545454545454

macro
Precision: 0.23593073593073594
Recall: 0.14098448368111288
F1 Score: 0.1443626630980261

weighted
Precision: 0.36170798898071627
Recall: 0.15454545454545454
F1 Score: 0.20206514796678732

s

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.06896551724137931

macro
Precision: 0.16666666666666666
Recall: 0.04
F1 Score: 0.06451612903225806

weighted
Precision: 0.28735632183908044
Recall: 0.06896551724137931
F1 Score: 0.11123470522803114

start XGB 1 VNIFGSM
(142, 56) (142,)
Accuracy: 0.2112676056338028

macro
Precision: 0.2522840531561462
Recall: 0.18736897274633124
F1 Score: 0.19530459421169802

weighted
Precision: 0.3479890973749474
Recall: 0.2112676056338028
F1 Score: 0.2511751341890648

start RF 100 baseline
(327, 56) (327,)
Accuracy: 0.045871559633027525

macro
Precision: 0.2516722408026756
Recall: 0.055769230769230765
F1 Score: 0.04436118396403147

weighted
Precision: 0.4772176367708877
Recall: 0.045871559633027525
F1 Score: 0.07886745886340278

start RF 100 BIM
(38, 56) (38,)
Accuracy: 0.631578947368421

macro
Precision: 0.5
Recall: 0.3157894736842105
F1 Score: 0.3870967741935484

weighted
Precision: 1.0
Recall: 0.631578947368421
F1 Score: 0.7741935483870968

start RF 100 FGSM
(19, 56) (19,)
Accuracy: 0.3

...

Accuracy: 0.7236363636363636

macro
Precision: 0.5244752830709749
Recall: 0.5664604424446944
F1 Score: 0.5039870894247199

weighted
Precision: 0.8733815369937988
Recall: 0.7236363636363636
F1 Score: 0.7836498265357197

start DT 100 FGSM
(1103, 56) (1103,)
Accuracy: 0.5203989120580236

macro
Precision: 0.4891611531502978
Recall: 0.32217527895135956
F1 Score: 0.3456531222629804

weighted
Precision: 0.9594874105074649
Recall: 0.5203989120580236
F1 Score: 0.6733781210189473

start DT 100 PGD
(825, 56) (825,)
Accuracy: 0.7236363636363636

macro
Precision: 0.5244752830709749
Recall: 0.5664604424446944
F1 Score: 0.5039870894247199

weighted
Precision: 0.8733815369937988
Recall: 0.7236363636363636
F1 Score: 0.7836498265357197

start DT 100 DF
(5683, 56) (5683,)
Accuracy: 0.2600739046278374

macro
Precision: 0.48877004965697174
Recall: 0.17012027411309494
F1 Score: 0.20686196562495385

weighted
Precision: 0.9702943775497077
Recall: 0.2600739046278374
F1 Score: 0.40955173599802996

start DT 100 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.784

macro
Precision: 0.541071369921503
Recall: 0.6092171533377972
F1 Score: 0.5366062029931806

weighted
Precision: 0.8978044032800166
Recall: 0.784
F1 Score: 0.8309506476071334

start DT 100 VNIFGSM
(755, 56) (755,)
Accuracy: 0.7708609271523179

macro
Precision: 0.5177267465158734
Recall: 0.5413636363636364
F1 Score: 0.5077095292117849

weighted
Precision: 0.8734381768219882
Recall: 0.7708609271523179
F1 Score: 0.8151962170227337

start DT 50 baseline
(354, 56) (354,)
Accuracy: 0.3615819209039548

macro
Precision: 0.4085232252410167
Recall: 0.47132554945054944
F1 Score: 0.30378028404344193

weighted
Precision: 0.4233302138574053
Recall: 0.3615819209039548
F1 Score: 0.2505121088974791

start DT 50 BIM
(1642, 56) (1642,)
Accuracy: 0.8032886723507917

macro
Precision: 0.5372604532356902
Recall: 0.5960181582360571
F1 Score: 0.5352375444177172

weighted
Precision: 0.9019113011101925
Recall: 0.8032886723507917
F1 Score: 0.8452052480901259

start DT 50 FGSM
(2808, 56) (2808,)
Ac

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8991150442477877

macro
Precision: 0.5
Recall: 0.4495575221238938
F1 Score: 0.47343895619757687

weighted
Precision: 1.0
Recall: 0.8991150442477877
F1 Score: 0.9468779123951538

start DT 1 SINIFGSM
(409, 56) (409,)
Accuracy: 0.48166259168704156

macro
Precision: 0.37056232526243604
Recall: 0.2977456313880646
F1 Score: 0.3291762084545398

weighted
Precision: 0.6060948900825998
Recall: 0.48166259168704156
F1 Score: 0.5364012933602473

start DT 1 VNIFGSM
(401, 56) (401,)
Accuracy: 0.2793017456359102

macro
Precision: 0.28295759928336817
Recall: 0.23479781088476742
F1 Score: 0.2478501009235642

weighted
Precision: 0.372264881242698
Recall: 0.2793017456359102
F1 Score: 0.31343828684809066



In [15]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.04075235109717868

macro
Precision: 0.232475715410869
Recall: 0.05307017543859649
F1 Score: 0.03961038961038961

weighted
Precision: 0.4399965104919795
Recall: 0.04075235109717868
F1 Score: 0.06961283230875707

start BIM
Accuracy: 0.1725254394079556

macro
Precision: 0.2182083178629896
Recall: 0.19804739724845818
F1 Score: 0.17158097759329524

weighted
Precision: 0.2718693066702186
Recall: 0.1725254394079556
F1 Score: 0.1842859296754388

start FGSM
Accuracy: 0.2641509433962264

macro
Precision: 0.24895412844036696
Recall: 0.28113562488802435
F1 Score: 0.25083240843507215

weighted
Precision: 0.24467841440193874
Recall: 0.2641509433962264
F1 Score: 0.24040374426737585

start PGD
Accuracy: 0.1725254394079556

macro
Precision: 0.2182083178629896
Recall: 0.19804739724845818
F1 Score: 0.17158097759329524

weighted
Precision: 0.2718693066702186
Recall: 0.1725254394079556
F1 Score: 0.1842859296754388

start DF
Accuracy: 0.18365119783817493

macro
Precision: 0.458666

In [15]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student_model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")